In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Load an image for prediction (for example, from a file)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import cv2

import numpy as np

import albumentations as A
from albumentations.pytorch import ToTensorV2

import os
import cv2
import torch
import timm
import time
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip "/content/drive/MyDrive/split.zip" -d "/content/LFR_split"


Streaming output truncated to the last 5000 lines.
  inflating: /content/LFR_split/split/val/left/1377375_69.jpg  
  inflating: /content/LFR_split/split/val/left/1377375_74.jpg  
  inflating: /content/LFR_split/split/val/left/1377375_9.jpg  
  inflating: /content/LFR_split/split/val/left/1377431_1.jpg  
  inflating: /content/LFR_split/split/val/left/1377431_4.jpg  
  inflating: /content/LFR_split/split/val/left/1378589_27.jpg  
  inflating: /content/LFR_split/split/val/left/1379938_10.jpg  
  inflating: /content/LFR_split/split/val/left/1380166_2.jpg  
  inflating: /content/LFR_split/split/val/left/1380193_0.jpg  
  inflating: /content/LFR_split/split/val/left/1380193_1.jpg  
  inflating: /content/LFR_split/split/val/left/1382250_0.jpg  
  inflating: /content/LFR_split/split/val/left/1382356_3.jpg  
  inflating: /content/LFR_split/split/val/left/1382356_8.jpg  
  inflating: /content/LFR_split/split/val/left/1382427_11.jpg  
  inflating: /content/LFR_split/split/val/left/1382427_15.jpg 

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_transforms(split="train"):
    if split == "train":
        return A.Compose([
            A.Resize(128, 128),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.GaussNoise(p=0.3),
            A.Blur(blur_limit=3, p=0.2),
            A.Rotate(limit=15, p=0.5),
            A.RandomGamma(p=0.3),
            A.GridDistortion(p=0.2),
            A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.5),
            A.CLAHE(p=0.3),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(128, 128),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ])


In [5]:
from torch.utils.data import Dataset
import os
import cv2

class LFRDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.samples = []

        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            for img_name in os.listdir(cls_path):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.samples.append((os.path.join(cls_path, img_name), self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)['image']
        return image, label


In [6]:
!ls "/content/LFR_split"


split


In [7]:

from torch.utils.data import DataLoader
import os

# Adjust path to your dataset location in Google Drive
base_path = "/content/LFR_split/split"


# Create datasets
train_dataset = LFRDataset(os.path.join(base_path, "train"), transform=get_transforms("train"))
val_dataset   = LFRDataset(os.path.join(base_path, "val"), transform=get_transforms("val"))
test_dataset  = LFRDataset(os.path.join(base_path, "test"), transform=get_transforms("val"))

# Create data loaders (keep num_workers low in Colab)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Print dataset sizes
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")


Train: 98506, Val: 21108, Test: 21111


<ipython-input-4-396266507bf4>:15: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.5),


In [8]:
# Install timm if not already installed
!pip install timm -q

import torch
import torch.nn as nn
import timm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pretrained EfficientNet-B0 from TIMM
model = timm.create_model('efficientnet_b0', pretrained=True)

# Freeze the base model (optional)
for param in model.parameters():
    param.requires_grad = False  # You can unfreeze later for fine-tuning

# Replace the classifier head for 3-class output (front, left, right)
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier.in_features, 3)  # 3 output classes
)

# Send model to GPU/CPU
model = model.to(device)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
class_labels = {"front": 0, "left": 1, "right": 2}
idx_to_class = {v: k for k, v in class_labels.items()}


In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4, weight_decay=1e-5)


In [11]:
def evaluate_model(model, dataloader, device=device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return accuracy


In [12]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device=device):
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch [{epoch+1}/{num_epochs}]")
        start_time = time.time()

        model.train()
        train_loss, train_correct = 0.0, 0
        train_bar = tqdm(train_loader, desc="Training", leave=False)

        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_bar.set_postfix(loss=loss.item())

        train_acc = train_correct / len(train_loader.dataset)
        avg_train_loss = train_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc="Validating", leave=False)
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_bar.set_postfix(loss=loss.item())

        val_acc = val_correct / len(val_loader.dataset)
        avg_val_loss = val_loss / len(val_loader.dataset)

        print(f"Epoch {epoch+1} done in {time.time() - start_time:.1f}s — "
              f"Train Loss: {avg_train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("✅ New best model saved.")

    print("Training complete.")


In [13]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=15)



Epoch [1/15]


Epoch 1 done in 242.0s — Train Loss: 0.7866, Acc: 0.6611 | Val Loss: 0.6424, Acc: 0.7231
✅ New best model saved.

Epoch [2/15]


Epoch 2 done in 222.8s — Train Loss: 0.7166, Acc: 0.6861 | Val Loss: 0.6175, Acc: 0.7260
✅ New best model saved.

Epoch [3/15]


Epoch 3 done in 216.7s — Train Loss: 0.7041, Acc: 0.6911 | Val Loss: 0.6043, Acc: 0.7264
✅ New best model saved.

Epoch [4/15]


Epoch 4 done in 224.2s — Train Loss: 0.7000, Acc: 0.6920 | Val Loss: 0.5979, Acc: 0.7292
✅ New best model saved.

Epoch [5/15]


...

Epoch 11 done in 220.9s — Train Loss: 0.6835, Acc: 0.6966 | Val Loss: 0.5805, Acc: 0.7348

Epoch [12/15]


Epoch 12 done in 213.8s — Train Loss: 0.6843, Acc: 0.6962 | Val Loss: 0.5793, Acc: 0.7341

Epoch [13/15]


Epoch 13 done in 213.2s — Train Loss: 0.6829, Acc: 0.6970 | Val Loss: 0.5814, Acc: 0.7384
✅ New best model saved.

Epoch [14/15]


Epoch 14 done in 213.9s — Train Loss: 0.6844, Acc: 0.6962 | Val Loss: 0.5805, Acc: 0.7356

Epoch [15/15]


Epoch 15 done in 213.1s — Train Loss: 0.6836, Acc: 0.6974 | Val Loss: 0.5810, Acc: 0.7359
Training complete.


In [14]:
# Save the model
model_path = "final_model.pth"
torch.save(model.state_dict(), model_path)

# Download the model
files.download(model_path)



NameError: name 'files' is not defined